# Weekly Meeting 3
---

1. I reproduced the experimentation both with the generative and extractive systems of the paper and I integrated my code for fast inference.
<br>

2. Number of seen and unseen entities:
        {"test": {"seen": 302, "unseen": 128} (70%/30%),
         "valid": {"seen": 305, "unseen": 125 (71%/29%)} 
<br>
                                     
## Error Analysis (Generative)
    
Same errors that I showed you in the previous meeting
    

    
Worse with locations, especially with unseen locations (the discriminative is also worse with locations than with persons and organizations)



General aggregations are quite confused: men, countries, states, nations, americans, writers ...


    
The discriminative model does not make that confusions with general aggregations:
    
                 Agg                  Gen                 Disc               NRanks

                 men                  102                    1                   13
           countries                  118                    1                   44
             nations                  111                    6                   40
              states                   60                    0                    7
           americans                   61                    6                   23
             writers                   52                    2                   12
             authors                   44                    1                    5
             leaders                   33                    0                    3
             political powers          39                    3                   41
             regions                   34                    1                    5
             groups                    14                    0                    1
             medical foundations       12                    0                    1
             foundations               11                    0                    1
    

    
But the discriminative model confuses other aggregations (I think more specific):
    
                 Agg                  Gen                 Disc               NRanks
                 
              people involved.t.c       0                    8                    5
              creators                  2                   10                    2
              american entertainers     1                    6                    3
              musical entertainers      0                    4                    1
              conflicting leaders       0                    3                    1
              european locations        2                   17                    1
              european places           3                   16                    1
              asian places              8                   19                    1
              areas involved i.t.d      4                   14                    1
              eastern countries         3                   10                    1
              news agencies             4                    7                    1
              corporations              1                    3                    3
              transportation companies  0                    1                    1

---

Why men, countries and nations? -> why are used these general aggregations? is it because it's difficult
      to aggregate some examples and they are used as "wildcard"? -> It's a mix, in some cases, "men" is not used
      as a wildcard because the annotators identified more specific aggregations:
      

    Entities (person): Nanni Moretti, Silvio Berlusconi
    8: public figures [0.013/1]
    13: men [0.009/1]
    16: italian [0.005/1]
    17: media participants [0.003/1]

    Entities (person): Louis J. Eppolito, Thomas Galpine
    1: luchese crime family participants [0.046/1]
    2: men on the plaintiff's side [0.038/1]
    6: men [0.021/1]

    Entities (person): Albert Einstein, Wolfgang Amadeus Mozart
    1: geniuses [0.054/1]
    3: historical figures [0.023/1]
    5: men [0.013/1]
    9: famous people [0.010/1]

    Entities (person): John D. Negroponte, Pat Roberts
    1: politicians [0.047/1]
    3: american public servants [0.042/1]
    9: statesmen [0.014/1]
    13: men [0.009/1]

        
But, in other cases, it's complex to identify more specific aggregations and it acts like a wildcard 
      to increase the number of gold aggregations:
        
        
    Entities (person): Robert Caro, Robert Moses 
    (the name is the only plausible aggregation)
    6: american men named robert [0.016/1]
    8: men [0.011/1]

    Entities (person): Billy Burt Hopper, Bobby Y. Emory 
    (Hopper is a sherif and Emory a criminal)
    3: convicted criminals [0.023/1]
    7: men [0.015/1]

    Entities (person): Edward L. Glaeser, Jon Gertner 
    (Glaeser is an economy professor and Gertner an historian)
    1: economists [0.051/1]
    11: men [0.004/1]

    Entities (person): John J. Doherty, Martin F. Horn, Michael R. Bloomberg 
    (it's quite difficult to distinguish an aggregation in the context. Also, only Bloomberg has background)
    3: new york city mayoral employees [0.027/1]
    11: men [0.007/1]

    Entities (person): Charles M. Kavanagh, Daniel Donohue
    (church member and a student abused by Kavanagh)
    3: church members [0.016/1]
    14: men [0.007/1]
      
In almost any ranking with "men" as gold aggregation, the word "man" does not appear in the aggregatable instance. So, the system should be able to infer it e.g. from male forms of nouns or pronouns


The use of "countries" as wildcard is frequent. The aggregation "countries" is even more frequent than the "men" aggregation (44 of 155 location rankings vs 13 of 248 person rankings):



    Entities (location): Iran, Syria
    4: nations [0.030/1]
    7: countries [0.023/1]
    There are more specific aggregations for that example (possibly fast annotation)
    (confused by very reasonable aggregations: 1: western asian regions [0.051/0] 2: western asian areas [0.044/0])

    Entities (location): Kosovo, Serbia:
    6: countries [0.022/1]
    7: nations [0.019/1]
    8: republics [0.017/1]
    Also it seems a fast annotation.

    Entities (location): Canada, Turkey
    4: nations [0.052/1]
    5: countries [0.049/1]
    I think general aggregations are suited for this example.

    Entities (location): Israel, Lebanon
    2: middle eastern countries [0.063/1]
    5: nations [0.033/1]
    6: countries [0.031/1]
    I think there are also more specific aggregations (e.g. western asia)
    
    
The word "country" appears exactly, in almost all cases, in the wikipedia backgrounds:

---

Is the generative system failing on detecting exact matches or lexical patterns? I thought this is related to the self-attention layers of the decoder.

Possibly the discriminative system is able to access better to exact words because the decoder also receives the text of the aggregatable instance instead of only the gold aggregation. I hypothesized that relating the gold aggregation with the aggregatable instance in all the self-attention layers of the decoder (in addition to the encoder-decoder attention with the contextualized encoder output) is benefitial to detect lexical patterns and exact matches (as the uncontextual embeddings are considered in the first layer of the decoder).  I made an experiment to test this with the generative model. Basically to train the generative system to generate "gold aggregation | aggregatable_instance" (v2 target_format,). It's basically use the same input/output than the discriminative approach (without negative candidates) to finetune BART, and in inference, compute the probability of the sequence gold aggregation | aggregatable_instance. Modified beam search to efficiently generate aggregations until the separator symbol | during generation:

(other ways to do this: modify BART to use the Q, K, V at all levels instead of the last contextualized):


> * TESA Paper (Generative)
> * average_precision: 0.70564 (+/-0.25775)
> * precision_at_10: 0.24209 (+/-0.08664)
> * recall_at_10: 0.90671 (+/-0.18503)
> * ndcg_at_10: 0.79057 (+/-0.22204)
> * reciprocal_best_rank: 0.84009 (+/-0.28662)
> * reciprocal_average_rank: 0.35655 (+/-0.22191)

> * Generative TF_V2
> * average_precision: 0.82907 (+/-0.21078)
> * precision_at_10: 0.25023 (+/-0.08964)
> * recall_at_10: 0.93074 (+/-0.16163)
> * ndcg_at_10: 0.88178 (+/-0.17535)
> * reciprocal_best_rank: 0.93693 (+/-0.19078)
> * reciprocal_average_rank: 0.42594 (+/-0.22274


It shows better behaviour (12+ average precision, 3+ recall and 9+ rbr). Also it's the best system detecting
locations (very similar to the discriminative).


The confusions with men, countries, nations, are drastically reduced:

* "men" confused by other 2 choices (appears in 13 rankings)
* "countries" confused by other 2 choices (appears in 44 rankings)
* "nations" confused by other 2 choices (appears in 40 rankings)

Now, the model confuses very specific aggregations. These aggregations appear in few rankings but they are very lower in the rankings:

* "masjid al-taqwa attendees" confused by other 21 choices (appears in 1 rankings)
* "interested in the washington redskins" confused by other 21 choices (appears in 1 rankings)
* "participants in a crimal case" confused by other 21 choices (appears in 1 rankings)
* "men who's first name begins with the letter r" confused by other 21 choices (appears in 1 rankings)
* "former mlb professional baseball pichers" confused by other 21 choices (appears in 1 rankings)
* "involved in a potentially criminal scheme" confused by other 20 choices (appears in 1 rankings)
* "american men named robert" confused by other 19 choices (appears in 1 rankings)

Same factual inconsistencies in generation than the generative system of the paper (american politicians, but lower ranked):


> 0. former French politicians | [0.059]
> 1. french politicians | [0.057]
> 2. politicians | [0.048]
> 3. american politicians | [0.044]
> 4. former french politicians | [0.042]
> 5. former French presidents | [0.039]
> 6. entertainers | [0.031]
> 7. politic figures | [0.030]
> 8. republicans | [0.029]
> 9. former French politician | [0.029]

### Evaluation with larger rankings (more distractors)

I evaluated the systems with 2x and 3x more distractors in the ranking (without training again with that number of candidates to make a fair comparison between the generative system and the discriminative - the generative is only trained with gold aggregations -):

      Average-Precision
---
      System                24 (1x)    48 (2x)              72 (3x)
                
      Generative           70.55     61.66 (Δ8.89)        57.78 (Δ12.77)  
      Discriminative       89.80     87.52 (Δ2.28)        83.00 (Δ6.80)
      Δ                    19.25     25.86                25.22


      Recall @10
---
      System                24 (1x)    48 (2x)              72 (3x)
                
      Generative           90.67     82.35 (Δ8.32)       77.11 (Δ13.56)
      Discriminative       99.28     97.69 (Δ1.59)       96.82 (Δ2.46)
      Δ                    8.61      15.34               19.71
      


## Hierarchy of concepts

Question: evaluate the generalization of the generated aggregations or the ranked aggregations?

DBPedia classes:
http://mappings.dbpedia.org/server/ontology/classes/
I think it is quite "flatten" person -> politician (leaf)
                                place -> settlement -> city (leaf)
                                organization -> company (leaf)
                           
                                
Wordnet *:

hypernyms and hyponyms, I think it's quite suitable for the task (visual inspection). Some specializations
are not synsets, but it is quite lexical -> american politicians, renowned artists, convicted criminals, ... (Qualifier+Noun) (convicted criminals <- criminals <- person) and the noun are synsets.
https://www.cs.princeton.edu/courses/archive/spring07/cos226/assignments/wordnet.html


Wikipedia:

Api: https://en.wikipedia.org/w/api.php?action=query&list=categorymembers&cmnamespace=14&cmlimit=max&cmtitle=Category:main%20topic%20classifications

Some human artifacts (members, list, .. by location, .. by location and occupation, ...
I'm preparing the data for generate the "modified" graph from wikipedia.



Evaluate the generalization on the hierarchy:

Traverse the graph to compute distances that measure the generalization given a generated aggregation and a gold
standard. Several measures, mostly based in the shortest path and information content are implemented in WordNet -> http://www.nltk.org/howto/wordnet.html, but:

* path_similarity("politician", "dog") = 0.14285714285714285
* path_similarity("politician", "president") = 0.14285714285714285